In [1]:
!pip install python-sql


     ---------------------------------------- 40.1/40.1 kB 1.9 MB/s eta 0:00:00


In [2]:
from sql import *
from sql.aggregate import *
from sql.conditionals import  *

In [3]:
user = Table('user')
select=user.select()
tuple(select)

('SELECT * FROM "user" AS "a"', ())

In [4]:
select=user.select(user.name)
tuple(select)

('SELECT "a"."name" FROM "user" AS "a"', ())

In [5]:
select=user.select(Count(Literal(1)))
tuple(select)

('SELECT COUNT(%s) FROM "user" AS "a"', (1,))

In [6]:
select=user.select(user.name,distinct=True)
tuple(select)

('SELECT DISTINCT "a"."name" FROM "user" AS "a"', ())

In [7]:
select=user.select(user.id,user.name)
tuple(select)

('SELECT "a"."id", "a"."name" FROM "user" AS "a"', ())

In [8]:
#select with where conditions:
select.where = user.name =='foo'
tuple(select)

('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE ("a"."name" = %s)',
 ('foo',))

In [9]:
select.where = (user.name =='foo')& (user.active==True)
tuple(select)

('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE (("a"."name" = %s) AND ("a"."active" = %s))',
 ('foo', True))

In [10]:
select.where = user.name ==user.login
tuple(select)

('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE ("a"."name" = "a"."login")',
 ())

In [11]:
#select with join:
join=user.join(Table('user_group'))
join.condition=join.right.useer==user.id
select=join.select(user.name,join.right.group)
tuple(select)

('SELECT "a"."name", "b"."group" FROM "user" AS "a" INNER JOIN "user_group" AS "b" ON ("b"."useer" = "a"."id")',
 ())

In [12]:
#select with multiple joins:
join1=user.join(Table('user'))
join2=join1.join(Table('user'))
select=join2.select(user.id,join1.right.id,join2.right.id)
tuple(select)

('SELECT "a"."id", "b"."id", "c"."id" FROM "user" AS "a" INNER JOIN "user" AS "b" INNER JOIN "user" AS "c"',
 ())

In [13]:
#select with groupby:
invoice=Table('invoice')
select=invoice.select(Sum(invoice.amount),invoice.currency,
       group_by=invoice.currency)
tuple(select)

('SELECT SUM("a"."amount"), "a"."currency" FROM "invoice" AS "a" GROUP BY "a"."currency"',
 ())

In [14]:
#select with output name:
tuple(user.select(user.name.as_('First Name')))

('SELECT "a"."name" AS "First Name" FROM "user" AS "a"', ())

In [15]:
#select with order_by
tuple(user.select(order_by=user.date))

('SELECT * FROM "user" AS "a" ORDER BY "a"."date"', ())

In [16]:
tuple(user.select(order_by=Asc(user.date)))

('SELECT * FROM "user" AS "a" ORDER BY "a"."date" ASC', ())

In [17]:
tuple(user.select(order_by=(user.date.asc, user.id.desc)))

('SELECT * FROM "user" AS "a" ORDER BY "a"."date" ASC, "a"."id" DESC', ())

In [18]:
#select with sub-select:
user_group=Table('user_group')
subselect=user_group.select(user_group.user,
    where=user_group.active==True)
user=Table('user')
tuple(user.select(user.id,where=user.id.in_(subselect)))

('SELECT "a"."id" FROM "user" AS "a" WHERE ("a"."id" IN (SELECT "b"."user" FROM "user_group" AS "b" WHERE ("b"."active" = %s)))',
 (True,))

In [19]:
tuple(subselect.select(subselect.user))

('SELECT "a"."user" FROM (SELECT "b"."user" FROM "user_group" AS "b" WHERE ("b"."active" = %s)) AS "a"',
 (True,))

In [20]:
other_table=Table('user','myschema')
tuple(other_table.select())

('SELECT * FROM "myschema"."user" AS "a"', ())

In [21]:
#insert query wit default valuse:
tuple(user.insert())
tuple(user.insert(columns=[user.name,user.login],
       values=[['Foo','foo']]))

('INSERT INTO "user" AS "a" ("name", "login") VALUES (%s, %s)', ('Foo', 'foo'))

In [22]:
tuple(user.insert(columns=[user.name,user.login],
       values=[['Foo','foo'],['Bar','bar']]))

('INSERT INTO "user" AS "a" ("name", "login") VALUES (%s, %s), (%s, %s)',
 ('Foo', 'foo', 'Bar', 'bar'))

In [23]:
#insert query with query:
passwd=Table('passwd')
sselect=passwd.select(passwd.login,passwd.passwd)
tuple(user.insert(values=select))

('INSERT INTO "user" AS "b" SELECT SUM("a"."amount"), "a"."currency" FROM "invoice" AS "a" GROUP BY "a"."currency"',
 ())

In [24]:
#update query with values:
tuple(user.update(columns=[user.active],values=[True]))
tuple(select)

('SELECT SUM("a"."amount"), "a"."currency" FROM "invoice" AS "a" GROUP BY "a"."currency"',
 ())

In [25]:
tuple(invoice.update(columns=[invoice.total],values=[invoice.amount + invoice.tax]))

('UPDATE "invoice" AS "a" SET "total" = ("a"."amount" + "a"."tax")', ())

In [26]:
#update query with where condition:
tuple(user.update(columns=[user.active],values=[True],
          where=user.active==False))

('UPDATE "user" AS "a" SET "active" = %s WHERE ("a"."active" = %s)',
 (True, False))